In [ ]:
#| default_exp losses
#| all_polars

# Losses

> Loss functions for model evaluation.
> 

The most important train signal is the forecast error, which is the difference between the observed value $y_{\tau}$ and the prediction $\hat{y}_{\tau}$, at time $y_{\tau}$:

$$
e_{\tau} = y_{\tau}-\hat{y}_{\tau} \qquad \qquad \tau \in \{t+1,\dots,t+H \}
$$

The train loss summarizes the forecast errors in different evaluation metrics.

In [ ]:
#| export
from typing import Callable, Dict, List, Optional, Tuple, Union
from numpy.typing import ArrayLike

import numpy as np
import pandas as pd

import utilsforecast.processing as ufp
from utilsforecast.compat import DFType, DataFrame, pl_DataFrame, pl, pl_Expr

In [ ]:
#| hide
import re
import warnings

from nbdev import show_doc

from utilsforecast.compat import POLARS_INSTALLED

In [ ]:
#| hide
warnings.filterwarnings('ignore', message='Unknown section References')

In [ ]:
from utilsforecast.data import generate_series

In [ ]:
#| polars
models = ['model0', 'model1']
series = generate_series(10, n_models=2, level=[80])
series_pl = generate_series(10, n_models=2, level=[80], engine='polars')

## 1. Scale-dependent Errors

### Mean Absolute Error (MAE)

$$
\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} |y_{\tau} - \hat{y}_{\tau}|
$$

![](imgs/losses/mae_loss.png)

In [ ]:
#| exporti
def _base_docstring(*args, **kwargs) -> Callable:
    base_docstring = """

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, actual values and predictions.
    models : list of str
        Columns that identify the models predictions.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.
    """
    def docstring_decorator(f: Callable):
        if f.__doc__ is not None:
            f.__doc__ += base_docstring
        return f

    return docstring_decorator(*args, **kwargs)

In [ ]:
#| exporti
def _pl_agg_expr(
    df: pl_DataFrame,
    models: Union[List[str], List[Tuple[str, str]]],
    id_col: str,
    gen_expr: Callable[[Union[str, Tuple[str, str]]], 'pl.Expr'],
) -> pl_DataFrame:
    exprs = [gen_expr(model) for model in models]
    df = df.select([id_col, *exprs])
    return ufp.group_by(df, id_col, maintain_order=True).mean()


def _scale_loss(
    loss_df: DFType,
    scale_type: str,
    models: List[str],
    seasonality: int,
    train_df: DFType,
    id_col: str = "unique_id",
    target_col: str = "y",
) -> DFType:
    """
    Parameters
    ----------
    loss_df : pandas or polars DataFrame
        Input dataframe with id, actuals, predictions and losses results.
    scale_type : str
        Type of scaling. Possible values are 'absolute_error' or 'squared_error'.
    models : list of str
        Columns that identify the models predictions.
    seasonality : int
        Main frequency of the time series;
        Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1.
    train_df : pandas or polars DataFrame
        Training dataframe with id and actual values. Must be sorted by time.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.
    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.
    References
    ----------
    [1] https://robjhyndman.com/papers/mase.pdf
    """

    if isinstance(train_df, pd.DataFrame):
        loss_df = loss_df.set_index(id_col)
        # assume train_df is sorted
        lagged = train_df.groupby(id_col, observed=True)[target_col].shift(seasonality)
        if scale_type == "absolute_error":
            scale = train_df[target_col].sub(lagged).abs()
        else:
            scale = train_df[target_col].sub(lagged).pow(2)
        scale = scale.groupby(train_df[id_col], observed=True).mean()
        res = loss_df.div(_zero_to_nan(scale), axis=0).fillna(0)
        res.index.name = id_col
        res = res.reset_index()
    else:
        # assume train_df is sorted
        lagged = pl.col(target_col).shift(seasonality).over(id_col)
        if scale_type == "absolute_error":
            scale_expr = pl.col(target_col).sub(lagged).abs().alias("scale")
        else:
            scale_expr = pl.col(target_col).sub(lagged).pow(2).alias("scale")
        scale = train_df.select([id_col, scale_expr])
        scale = ufp.group_by(scale, id_col).mean()
        scale = scale.with_columns(_zero_to_nan(pl.col("scale")))

        def gen_expr(model):
            return pl.col(model).truediv(pl.col("scale")).fill_nan(0).alias(model)

        full_df = loss_df.join(scale, on=id_col, how="left")
        res = _pl_agg_expr(full_df, models, id_col, gen_expr)

    return res

In [ ]:
#| export
@_base_docstring
def mae(
    df: DFType,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Mean Absolute Error (MAE)

    MAE measures the relative prediction
    accuracy of a forecasting method by calculating the
    deviation of the prediction and the true
    value at a given time and averages these devations
    over the length of the series."""
    if isinstance(df, pd.DataFrame):
        res = (df[models].sub(df[target_col], axis=0)).abs().groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        def gen_expr(model):
            return pl.col(target_col).sub(pl.col(model)).abs().alias(model)

        res = _pl_agg_expr(df, models, id_col, gen_expr)
    return res

In [ ]:
show_doc(mae, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L129){target="_blank" style="float:right; font-size:smaller"}

#### mae

>      mae (df:~DFType, models:List[str], id_col:str='unique_id',
>           target_col:str='y')

*Mean Absolute Error (MAE)

MAE measures the relative prediction
accuracy of a forecasting method by calculating the
deviation of the prediction and the true
value at a given time and averages these devations
over the length of the series.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
def pd_vs_pl(pd_df, pl_df, models):
    np.testing.assert_allclose(
        pd_df[models].to_numpy(),
        pl_df.sort('unique_id').select(models).to_numpy(),
    )

In [ ]:
#| polars
pd_vs_pl(
    mae(series, models),
    mae(series_pl, models),
    models,
)

### Mean Squared Error

$$
\mathrm{MSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} (y_{\tau} - \hat{y}_{\tau})^{2}
$$

![](imgs/losses/mse_loss.png)

In [ ]:
#| export
@_base_docstring
def mse(
    df: DFType,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Mean Squared Error (MSE)
    
    MSE measures the relative prediction
    accuracy of a forecasting method by calculating the 
    squared deviation of the prediction and the true
    value at a given time, and averages these devations
    over the length of the series."""    
    if isinstance(df, pd.DataFrame):
        res = (df[models].sub(df[target_col], axis=0)).pow(2).groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        def gen_expr(model):
            return pl.col(target_col).sub(pl.col(model)).pow(2).alias(model)

        res = _pl_agg_expr(df, models, id_col, gen_expr)
    return res

In [ ]:
show_doc(mse, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L161){target="_blank" style="float:right; font-size:smaller"}

#### mse

>      mse (df:~DFType, models:List[str], id_col:str='unique_id',
>           target_col:str='y')

*Mean Squared Error (MSE)

MSE measures the relative prediction
accuracy of a forecasting method by calculating the 
squared deviation of the prediction and the true
value at a given time, and averages these devations
over the length of the series.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    mse(series, models),
    mse(series_pl, models),
    models,
)

### Root Mean Squared Error

$$
\mathrm{RMSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \sqrt{\frac{1}{H} \sum^{t+H}_{\tau=t+1} (y_{\tau} - \hat{y}_{\tau})^{2}}
$$

![](imgs/losses/rmse_loss.png)

In [ ]:
#| export
@_base_docstring
def rmse(
    df: DFType,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Root Mean Squared Error (RMSE)
    
    RMSE measures the relative prediction
    accuracy of a forecasting method by calculating the squared deviation
    of the prediction and the observed value at a given time and
    averages these devations over the length of the series.
    Finally the RMSE will be in the same scale
    as the original time series so its comparison with other
    series is possible only if they share a common scale. 
    RMSE has a direct connection to the L2 norm."""    
    res = mse(df, models, id_col, target_col)
    if isinstance(res, pd.DataFrame):
        res[models] = res[models].pow(0.5)
    else:
        res = res.with_columns(*[pl.col(c).pow(0.5) for c in models])
    return res

In [ ]:
show_doc(rmse, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L193){target="_blank" style="float:right; font-size:smaller"}

#### rmse

>      rmse (df:~DFType, models:List[str], id_col:str='unique_id',
>            target_col:str='y')

*Root Mean Squared Error (RMSE)

RMSE measures the relative prediction
accuracy of a forecasting method by calculating the squared deviation
of the prediction and the observed value at a given time and
averages these devations over the length of the series.
Finally the RMSE will be in the same scale
as the original time series so its comparison with other
series is possible only if they share a common scale. 
RMSE has a direct connection to the L2 norm.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    rmse(series, models),
    rmse(series_pl, models),
    models,
)

In [ ]:
#| export
@_base_docstring
def bias(
    df: DFType,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Forecast estimator bias.
    
    Defined as prediction - actual"""
    if isinstance(df, pd.DataFrame):
        res = (df[models].sub(df[target_col], axis=0)).groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        def gen_expr(model):
            return pl.col(model).sub(pl.col(target_col)).alias(model)

        res = _pl_agg_expr(df, models, id_col, gen_expr)
    return res

In [ ]:
show_doc(bias, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L218){target="_blank" style="float:right; font-size:smaller"}

#### bias

>      bias (df:~DFType, models:List[str], id_col:str='unique_id',
>            target_col:str='y')

*Forecast estimator bias.

Defined as prediction - actual*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    bias(series, models),
    bias(series_pl, models),
    models,
)

## 2. Percentage Errors

### Mean Absolute Percentage Error

$$
\mathrm{MAPE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{|y_{\tau}|}
$$

![](imgs/losses/mape_loss.png)

In [ ]:
#| exporti
def _zero_to_nan(series: Union[pd.Series, 'pl.Expr']) -> Union[pd.Series, 'pl.Expr']:
    if isinstance(series, pd.Series):
        res = series.replace(0, np.nan)
    else:
        res = (
            pl.when(series == 0)
            .then(float('nan'))
            .otherwise(series.abs())
        )
    return res

In [ ]:
#| export
@_base_docstring
def mape(
    df: DFType,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Mean Absolute Percentage Error (MAPE)
    
    MAPE measures the relative prediction
    accuracy of a forecasting method by calculating the percentual deviation
    of the prediction and the observed value at a given time and
    averages these devations over the length of the series.
    The closer to zero an observed value is, the higher penalty MAPE loss
    assigns to the corresponding error."""
    if isinstance(df, pd.DataFrame):
        res = (
            df[models]
            .sub(df[target_col], axis=0)
            .abs()
            .div(_zero_to_nan(df[target_col].abs()), axis=0)
            .groupby(df[id_col], observed=True).mean()
        )
        res.index.name = id_col
        res = res.reset_index()
    else:
        def gen_expr(model):
            abs_err = pl.col(target_col).sub(pl.col(model)).abs()
            abs_target = _zero_to_nan(pl.col(target_col))
            ratio = abs_err.truediv(abs_target).alias(model)
            return ratio.fill_nan(None)

        res = _pl_agg_expr(df, models, id_col, gen_expr)
    return res

In [ ]:
show_doc(mape, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L253){target="_blank" style="float:right; font-size:smaller"}

#### mape

>      mape (df:~DFType, models:List[str], id_col:str='unique_id',
>            target_col:str='y')

*Mean Absolute Percentage Error (MAPE)

MAPE measures the relative prediction
accuracy of a forecasting method by calculating the percentual deviation
of the prediction and the observed value at a given time and
averages these devations over the length of the series.
The closer to zero an observed value is, the higher penalty MAPE loss
assigns to the corresponding error.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    mape(series, models),
    mape(series_pl, models),
    models,
)

### Symmetric Mean Absolute Percentage Error

$$
\mathrm{SMAPE}_{2}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{|y_{\tau}|+|\hat{y}_{\tau}|}
$$

In [ ]:
#| export
@_base_docstring
def smape(
    df: DFType,
    models: List[str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Symmetric Mean Absolute Percentage Error (SMAPE)

    SMAPE measures the relative prediction
    accuracy of a forecasting method by calculating the relative deviation
    of the prediction and the observed value scaled by the sum of the
    absolute values for the prediction and observed value at a
    given time, then averages these devations over the length
    of the series. This allows the SMAPE to have bounds between
    0% and 100% which is desirable compared to normal MAPE that
    may be undetermined when the target is zero."""
    if isinstance(df, pd.DataFrame):
        delta_y = df[models].sub(df[target_col], axis=0).abs()
        scale = df[models].abs().add(df[target_col].abs(), axis=0)
        raw = delta_y.div(scale).fillna(0)
        res = raw.groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        def gen_expr(model):
            abs_err = pl.col(model).sub(pl.col(target_col)).abs()
            denominator = _zero_to_nan(pl.col(model).abs().add(pl.col(target_col)).abs())
            ratio = abs_err.truediv(denominator).alias(model)
            return ratio.fill_nan(0)

        res = _pl_agg_expr(df, models, id_col, gen_expr)
    return res

In [ ]:
show_doc(smape, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L291){target="_blank" style="float:right; font-size:smaller"}

#### smape

>      smape (df:~DFType, models:List[str], id_col:str='unique_id',
>             target_col:str='y')

*Symmetric Mean Absolute Percentage Error (SMAPE)

SMAPE measures the relative prediction
accuracy of a forecasting method by calculating the relative deviation
of the prediction and the observed value scaled by the sum of the
absolute values for the prediction and observed value at a
given time, then averages these devations over the length
of the series. This allows the SMAPE to have bounds between
0% and 100% which is desirable compared to normal MAPE that
may be undetermined when the target is zero.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    smape(series, models),
    smape(series_pl, models),
    models,
)

## 3. Scale-independent Errors

### Mean Absolute Scaled Error

$$
\mathrm{MASE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau}) = 
\frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau})}
$$

![](imgs/losses/mase_loss.png)

In [ ]:
#| export
def mase(
    df: DFType,
    models: List[str],
    seasonality: int,
    train_df: DFType,
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Mean Absolute Scaled Error (MASE)
    
    MASE measures the relative prediction
    accuracy of a forecasting method by comparinng the mean absolute errors
    of the prediction and the observed value against the mean
    absolute errors of the seasonal naive model.
    The MASE partially composed the Overall Weighted Average (OWA), 
    used in the M4 Competition.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, actuals and predictions.
    models : list of str
        Columns that identify the models predictions.
    seasonality : int
        Main frequency of the time series;
        Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1.
    train_df : pandas or polars DataFrame
        Training dataframe with id and actual values. Must be sorted by time.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://robjhyndman.com/papers/mase.pdf        
    """
    mean_abs_err = mae(df, models, id_col, target_col)
    return _scale_loss(
        loss_df=mean_abs_err,
        scale_type="absolute_error",
        models=models,
        seasonality=seasonality,
        train_df=train_df,
        id_col=id_col,
        target_col=target_col
    )

In [ ]:
show_doc(mase, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L328){target="_blank" style="float:right; font-size:smaller"}

#### mase

>      mase (df:~DFType, models:List[str], seasonality:int, train_df:~DFType,
>            id_col:str='unique_id', target_col:str='y')

*Mean Absolute Scaled Error (MASE)

MASE measures the relative prediction
accuracy of a forecasting method by comparinng the mean absolute errors
of the prediction and the observed value against the mean
absolute errors of the seasonal naive model.
The MASE partially composed the Overall Weighted Average (OWA), 
used in the M4 Competition.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actuals and predictions. |
| models | List |  | Columns that identify the models predictions. |
| seasonality | int |  | Main frequency of the time series;<br>Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1. |
| train_df | DFType |  | Training dataframe with id and actual values. Must be sorted by time. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    mase(series, models, 7, series),
    mase(series_pl, models, 7, series_pl),
    models,
)

### Relative Mean Absolute Error

$$
\mathrm{RMAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{base}_{\tau}) = \frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{|y_{\tau}-\hat{y}_{\tau}|}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{base}_{\tau})}
$$

![](imgs/losses/rmae_loss.png)

In [ ]:
#| export
def rmae(
    df: DFType,
    models: List[str],
    baseline: str,
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Relative Mean Absolute Error (RMAE)
    
    Calculates the RAME between two sets of forecasts (from two different forecasting methods).
    A number smaller than one implies that the forecast in the 
    numerator is better than the forecast in the denominator.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : list of str
        Columns that identify the models predictions.
    baseline : str
        Column that identifies the baseline model predictions.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.
    """
    numerator = mae(df, models, id_col, target_col)
    denominator = mae(df, [baseline], id_col, target_col)
    if ufp.is_nan(denominator[baseline]).any():
        raise ValueError(f'baseline model ({baseline}) contains NaNs.')
    denominator = ufp.rename(denominator, {baseline: f'{baseline}_denominator'})
    res = ufp.join(numerator, denominator, on=id_col)
    if isinstance(numerator, pd.DataFrame):
        for model in models:
            res[model] = res[model].div(_zero_to_nan(res[f'{baseline}_denominator'])).fillna(0)
        res = res[[id_col, *models]]
    else:
        def gen_expr(model, baseline) -> pl_Expr:
            denominator = _zero_to_nan(pl.col(f'{baseline}_denominator'))
            return pl.col(model).truediv(denominator).fill_nan(0).alias(model)

        exprs: List[pl_Expr] = [gen_expr(m, baseline) for m in models]
        res = res.select([id_col, *exprs])
    return res

In [ ]:
show_doc(rmae, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L382){target="_blank" style="float:right; font-size:smaller"}

#### rmae

>      rmae (df:~DFType, models:List[str], baseline:str, id_col:str='unique_id',
>            target_col:str='y')

*Relative Mean Absolute Error (RMAE)

Calculates the RAME between two sets of forecasts (from two different forecasting methods).
A number smaller than one implies that the forecast in the 
numerator is better than the forecast in the denominator.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, times, actuals and predictions. |
| models | List |  | Columns that identify the models predictions. |
| baseline | str |  | Column that identifies the baseline model predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    rmae(series, models, models[0]),
    rmae(series_pl, models, models[0]),
    models,
)

### Mean Squared Scaled Error

$$
\mathrm{MSSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau}) = 
\frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{(y_{\tau}-\hat{y}_{\tau})^2}{\mathrm{MSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau})}
$$

In [ ]:
#| export
def msse(
    df: DFType,
    models: List[str],
    seasonality: int,
    train_df: DFType,
    id_col: str = "unique_id",
    target_col: str = "y",
) -> DFType:
    """Mean Squared Scaled Error (MSSE)

    MSSE measures the relative prediction
    accuracy of a forecasting method by comparinng the mean squared errors
    of the prediction and the observed value against the mean
    squared errors of the seasonal naive model.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, actuals and predictions.
    models : list of str
        Columns that identify the models predictions.
    seasonality : int
        Main frequency of the time series;
        Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1.
    train_df : pandas or polars DataFrame
        Training dataframe with id and actual values. Must be sorted by time.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://otexts.com/fpp3/accuracy.html
    """
    mean_sq_err = mse(df=df, models=models, id_col=id_col, target_col=target_col)
    return _scale_loss(
        loss_df=mean_sq_err,
        scale_type="squared_error",
        models=models,
        seasonality=seasonality,
        train_df=train_df,
        id_col=id_col,
        target_col=target_col
    )

In [ ]:
show_doc(msse, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L436){target="_blank" style="float:right; font-size:smaller"}

#### msse

>      msse (df:~DFType, models:List[str], seasonality:int, train_df:~DFType,
>            id_col:str='unique_id', target_col:str='y')

*Mean Squared Scaled Error (MSSE)

MSSE measures the relative prediction
accuracy of a forecasting method by comparinng the mean squared errors
of the prediction and the observed value against the mean
squared errors of the seasonal naive model.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actuals and predictions. |
| models | List |  | Columns that identify the models predictions. |
| seasonality | int |  | Main frequency of the time series;<br>Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1. |
| train_df | DFType |  | Training dataframe with id and actual values. Must be sorted by time. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    msse(series, models, 7, series),
    msse(series_pl, models, 7, series_pl),
    models,
)

### Root Mean Squared Scaled Error

$$
\mathrm{RMSSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau}) = 
\sqrt{\frac{1}{H} \sum^{t+H}_{\tau=t+1} \frac{(y_{\tau}-\hat{y}_{\tau})^2}{\mathrm{MSE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau})}}
$$

In [ ]:
#| export
def rmsse(
    df: DFType,
    models: List[str],
    seasonality: int,
    train_df: DFType,
    id_col: str = "unique_id",
    target_col: str = "y",
) -> DFType:
    res = msse(
        df,
        models=models,
        seasonality=seasonality,
        train_df=train_df,
        id_col=id_col,
        target_col=target_col,
    )
    if isinstance(res, pd.DataFrame):
        res[models] = res[models].pow(0.5)
    else:
        res = res.with_columns(pl.col(models).pow(0.5))
    return res

rmsse.__doc__ = msse.__doc__.replace(  # type: ignore[union-attr]
    'Mean Squared Scaled Error (MSSE)', 'Root Mean Squared Scaled Error (RMSSE)'
)

In [ ]:
show_doc(rmsse, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L488){target="_blank" style="float:right; font-size:smaller"}

#### rmsse

>      rmsse (df:~DFType, models:List[str], seasonality:int, train_df:~DFType,
>             id_col:str='unique_id', target_col:str='y')

*Root Mean Squared Scaled Error (RMSSE)

MSSE measures the relative prediction
accuracy of a forecasting method by comparinng the mean squared errors
of the prediction and the observed value against the mean
squared errors of the seasonal naive model.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actuals and predictions. |
| models | List |  | Columns that identify the models predictions. |
| seasonality | int |  | Main frequency of the time series;<br>Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1. |
| train_df | DFType |  | Training dataframe with id and actual values. Must be sorted by time. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    rmsse(series, models, 7, series),
    rmsse(series_pl, models, 7, series_pl),
    models,
)

## 4. Probabilistic Errors

### Quantile Loss

$$
\mathrm{QL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q)}_{\tau}) = 
\frac{1}{H} \sum^{t+H}_{\tau=t+1} 
\Big( (1-q)\,( \hat{y}^{(q)}_{\tau} - y_{\tau} )_{+} 
+ q\,( y_{\tau} - \hat{y}^{(q)}_{\tau} )_{+} \Big)
$$

![](imgs/losses/q_loss.png)

In [ ]:
#| export
def quantile_loss(
    df: DFType,
    models: Dict[str, str],
    q: float = 0.5,
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Quantile Loss (QL)

    QL measures the deviation of a quantile forecast.
    By weighting the absolute deviation in a non symmetric way, the
    loss pays more attention to under or over estimation.    
    A common value for q is 0.5 for the deviation from the median.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to str
        Mapping from model name to the model predictions for the specified quantile.
    q : float (default=0.5)
        Quantile for the predictions' comparison.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.
    """
    if isinstance(df, pd.DataFrame):
        res: Optional[pd.DataFrame] = None
        for model_name, pred_col in models.items():
            delta_y = df[target_col].sub(df[pred_col], axis=0)
            model_res = (
                np.maximum(q * delta_y, (q - 1) * delta_y)
                .groupby(df[id_col], observed=True)
                .mean()
                .rename(model_name)
                .reset_index()
            )
            if res is None:
                res = model_res
            else:
                res[model_name] = model_res[model_name]
    else:
        def gen_expr(model):
            model_name, pred_col = model
            delta_y = pl.col(target_col).sub(pl.col(pred_col))
            try:
                col_max = pl.max_horizontal([q * delta_y, (q - 1) * delta_y])
            except AttributeError:
                col_max = pl.max([q * delta_y, (q - 1) * delta_y])
            return col_max.alias(model_name)

        res = _pl_agg_expr(df, list(models.items()), id_col, gen_expr)
    return res

In [ ]:
show_doc(quantile_loss, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L516){target="_blank" style="float:right; font-size:smaller"}

#### quantile_loss

>      quantile_loss (df:~DFType, models:Dict[str,str], q:float=0.5,
>                     id_col:str='unique_id', target_col:str='y')

*Quantile Loss (QL)

QL measures the deviation of a quantile forecast.
By weighting the absolute deviation in a non symmetric way, the
loss pays more attention to under or over estimation.    
A common value for q is 0.5 for the deviation from the median.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions for the specified quantile. |
| q | float | 0.5 | Quantile for the predictions' comparison. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| hide
df = pd.DataFrame({
    "unique_id": [0, 1, 2],
    "y": [1.0, 2.0, 3.0],
    "overestimation": [2.0, 3.0, 4.0], # y + 1.
    "underestimation": [0.0, 1.0, 2.0], # y - 1.
})
df["unique_id"] = df["unique_id"].astype("category")
df = pd.concat([df, df.assign(unique_id=2)]).reset_index(drop=True)

ql_models_test = ["overestimation", "underestimation"]
quantiles = np.array([0.1, 0.9])

for q in quantiles:
    ql_df = quantile_loss(df, models=dict(zip(ql_models_test, ql_models_test)), q=q)
    # for overestimation, delta_y = y - y_hat = -1 so ql = max(-q, -(q-1)) 
    assert all(max(-q, -(q - 1)) == ql for ql in ql_df["overestimation"])
    # for underestimation, delta_y = y - y_hat = 1, so ql = max(q, q-1)
    assert all(max(q, q - 1) == ql for ql in ql_df["underestimation"])

In [ ]:
#| hide
#| polars
q_models = {
    0.1: {
        'model0': 'model0-lo-80',
        'model1': 'model1-lo-80',
    },
    0.9: {
        'model0': 'model0-hi-80',
        'model1': 'model1-hi-80',
    },
}

for q in quantiles:
    pd_vs_pl(
        quantile_loss(series, q_models[q], q=q),
        quantile_loss(series_pl, q_models[q], q=q),
        models,
    )

### Scaled Quantile Loss

$$
\mathrm{SQL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q)}_{\tau}) = 
\frac{1}{H} \sum^{t+H}_{\tau=t+1} 
\frac{(1-q)\,( \hat{y}^{(q)}_{\tau} - y_{\tau} )_{+} 
+ q\,( y_{\tau} - \hat{y}^{(q)}_{\tau} )_{+}}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau})}
$$

In [ ]:
#| export
def scaled_quantile_loss(
    df: DFType,
    models: Dict[str, str],
    seasonality: int,
    train_df: DFType,    
    q: float = 0.5,
    id_col: str = "unique_id",
    target_col: str = "y",
) -> DFType:
    """Scaled Quantile Loss (SQL)

    SQL measures the deviation of a quantile forecast scaled by
    the mean absolute errors of the seasonal naive model.
    By weighting the absolute deviation in a non symmetric way, the
    loss pays more attention to under or over estimation.
    A common value for q is 0.5 for the deviation from the median.
    This was the official measure used in the M5 Uncertainty competition
    with seasonality = 1.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to str
        Mapping from model name to the model predictions for the specified quantile.
    seasonality : int
        Main frequency of the time series;
        Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1.
    train_df : pandas or polars DataFrame
        Training dataframe with id and actual values. Must be sorted by time.        
    q : float (default=0.5)
        Quantile for the predictions' comparison.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://www.sciencedirect.com/science/article/pii/S0169207021001722
    """     
    q_loss = quantile_loss(df=df, models=models, q=q, id_col=id_col, target_col=target_col)
    return _scale_loss(
        loss_df=q_loss,
        scale_type="absolute_error",
        models=list(models.keys()),
        seasonality=seasonality,
        train_df=train_df,
        id_col=id_col,
        target_col=target_col,
    )

In [ ]:
show_doc(scaled_quantile_loss)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L578){target="_blank" style="float:right; font-size:smaller"}

### scaled_quantile_loss

>      scaled_quantile_loss (df:~DFType, models:Dict[str,str], seasonality:int,
>                            train_df:~DFType, q:float=0.5,
>                            id_col:str='unique_id', target_col:str='y')

*Scaled Quantile Loss (SQL)

SQL measures the deviation of a quantile forecast scaled by
the mean absolute errors of the seasonal naive model.
By weighting the absolute deviation in a non symmetric way, the
loss pays more attention to under or over estimation.
A common value for q is 0.5 for the deviation from the median.
This was the official measure used in the M5 Uncertainty competition
with seasonality = 1.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions for the specified quantile. |
| seasonality | int |  | Main frequency of the time series;<br>Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1. |
| train_df | DFType |  | Training dataframe with id and actual values. Must be sorted by time.         |
| q | float | 0.5 | Quantile for the predictions' comparison. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| hide
#| polars
q_models = {
    0.1: {
        'model0': 'model0-lo-80',
        'model1': 'model1-lo-80',
    },
    0.9: {
        'model0': 'model0-hi-80',
        'model1': 'model1-hi-80',
    },
}

for q in quantiles:
    pd_vs_pl(
        scaled_quantile_loss(series, q_models[q], seasonality=1, train_df=series, q=q),
        scaled_quantile_loss(series_pl, q_models[q], seasonality=1, train_df=series_pl, q=q),
        models,
    )

### Multi-Quantile Loss

$$
\mathrm{MQL}(\mathbf{y}_{\tau},
[\mathbf{\hat{y}}^{(q_{1})}_{\tau}, ... ,\hat{y}^{(q_{n})}_{\tau}]) = 
\frac{1}{n} \sum_{q_{i}} \mathrm{QL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q_{i})}_{\tau})
$$

![](imgs/losses/mq_loss.png)

In [ ]:
#| export
def mqloss(
    df: DFType,
    models: Dict[str, List[str]],
    quantiles: np.ndarray,
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Multi-Quantile loss (MQL)
    
    MQL calculates the average multi-quantile Loss for
    a given set of quantiles, based on the absolute 
    difference between predicted quantiles and observed values.

    The limit behavior of MQL allows to measure the accuracy 
    of a full predictive distribution with 
    the continuous ranked probability score (CRPS). This can be achieved 
    through a numerical integration technique, that discretizes the quantiles 
    and treats the CRPS integral with a left Riemann approximation, averaging over 
    uniformly distanced quantiles.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to list of str
        Mapping from model name to the model predictions for each quantile.
    quantiles : numpy array
        Quantiles to compare against.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://www.jstor.org/stable/2629907
    """
    res: Optional[DataFrame] = None
    error = np.empty((df.shape[0], quantiles.size))
    for model, predictions in models.items():
        for j, q_preds in enumerate(predictions):
            error[:, j] = (df[target_col] - df[q_preds]).to_numpy()
        loss = np.maximum(error * quantiles, error * (quantiles - 1)).mean(axis=1)
        model_res = type(df)({id_col: df[id_col], model: loss})
        model_res = ufp.group_by_agg(
            model_res, by=id_col, aggs={model: 'mean'}, maintain_order=True
        )
        if res is None:
            res = model_res
        else:
            res = ufp.assign_columns(res, model, model_res[model])
    return res

In [ ]:
show_doc(mqloss, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L638){target="_blank" style="float:right; font-size:smaller"}

#### mqloss

>      mqloss (df:~DFType, models:Dict[str,List[str]], quantiles:numpy.ndarray,
>              id_col:str='unique_id', target_col:str='y')

*Multi-Quantile loss (MQL)

MQL calculates the average multi-quantile Loss for
a given set of quantiles, based on the absolute 
difference between predicted quantiles and observed values.

The limit behavior of MQL allows to measure the accuracy 
of a full predictive distribution with 
the continuous ranked probability score (CRPS). This can be achieved 
through a numerical integration technique, that discretizes the quantiles 
and treats the CRPS integral with a left Riemann approximation, averaging over 
uniformly distanced quantiles.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions for each quantile. |
| quantiles | ndarray |  | Quantiles to compare against. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| hide
#| polars
mq_models = {
    'model0': ['model0-lo-80', 'model0-hi-80'],
    'model1': ['model1-lo-80', 'model1-hi-80'],
}

expected = pd.concat(
    [
        quantile_loss(series, models=q_models[q], q=q)
        for i, q in enumerate(quantiles)
    ]
).groupby('unique_id', observed=True, as_index=False).mean()
actual = mqloss(
    series,
    models=mq_models,
    quantiles=quantiles,
)
pd.testing.assert_frame_equal(actual, expected)

In [ ]:
#| polars
pd_vs_pl(
    mqloss(series, mq_models, quantiles=quantiles),
    mqloss(series_pl, mq_models, quantiles=quantiles),
    models,
)

In [ ]:
#| hide
#| polars
for series_df in [series, series_pl]:
    if isinstance(series_df, pd.DataFrame):
        df_test = series_df.assign(unique_id=lambda df: df["unique_id"].astype(str))
    else:
        df_test = series_df.with_columns(pl.col("unique_id").cast(pl.Utf8))
    mql_df = mqloss(
        df_test,
        mq_models, 
        quantiles=quantiles,
    )
    assert mql_df.shape == (series["unique_id"].nunique(), 1 + len(models))
    if isinstance(mql_df, pd.DataFrame):
        null_vals = mql_df.isna().sum().sum()
    else:
        null_vals = series_df.select(pl.all().is_null().sum()).sum_horizontal()
    assert null_vals.item() == 0

### Scaled Multi-Quantile Loss

$$
\mathrm{MQL}(\mathbf{y}_{\tau},
[\mathbf{\hat{y}}^{(q_{1})}_{\tau}, ... ,\hat{y}^{(q_{n})}_{\tau}]) = 
\frac{1}{n} \sum_{q_{i}} \frac{\mathrm{QL}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{(q_{i})}_{\tau})}{\mathrm{MAE}(\mathbf{y}_{\tau}, \mathbf{\hat{y}}^{season}_{\tau})}
$$

In [ ]:
#| export
def scaled_mqloss(
    df: DFType,
    models: Dict[str, List[str]],
    quantiles: np.ndarray,
    seasonality: int,
    train_df: DFType,
    id_col: str = "unique_id",
    target_col: str = "y",
) -> DFType:
    """Scaled Multi-Quantile loss (SMQL)

    SMQL calculates the average multi-quantile Loss for
    a given set of quantiles, based on the absolute
    difference between predicted quantiles and observed values 
    scaled by the mean absolute errors of the seasonal naive model. 
    The limit behavior of MQL allows to measure the accuracy
    of a full predictive distribution with
    the continuous ranked probability score (CRPS). This can be achieved
    through a numerical integration technique, that discretizes the quantiles
    and treats the CRPS integral with a left Riemann approximation, averaging over
    uniformly distanced quantiles.
    This was the official measure used in the M5 Uncertainty competition
    with seasonality = 1.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to list of str
        Mapping from model name to the model predictions for each quantile.
    quantiles : numpy array
        Quantiles to compare against.
    seasonality : int
        Main frequency of the time series;
        Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1.
    train_df : pandas or polars DataFrame
        Training dataframe with id and actual values. Must be sorted by time.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://www.sciencedirect.com/science/article/pii/S0169207021001722
    """
    mq_loss = mqloss(df=df, models=models, quantiles=quantiles, id_col=id_col, target_col=target_col)
    return _scale_loss(
        loss_df=mq_loss,
        scale_type="absolute_error",
        models=list(models.keys()),
        seasonality=seasonality,
        train_df=train_df,
        id_col=id_col,
        target_col=target_col,
    )

In [ ]:
show_doc(scaled_mqloss)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L697){target="_blank" style="float:right; font-size:smaller"}

### scaled_mqloss

>      scaled_mqloss (df:~DFType, models:Dict[str,List[str]],
>                     quantiles:numpy.ndarray, seasonality:int,
>                     train_df:~DFType, id_col:str='unique_id',
>                     target_col:str='y')

*Scaled Multi-Quantile loss (SMQL)

SMQL calculates the average multi-quantile Loss for
a given set of quantiles, based on the absolute
difference between predicted quantiles and observed values 
scaled by the mean absolute errors of the seasonal naive model. 
The limit behavior of MQL allows to measure the accuracy
of a full predictive distribution with
the continuous ranked probability score (CRPS). This can be achieved
through a numerical integration technique, that discretizes the quantiles
and treats the CRPS integral with a left Riemann approximation, averaging over
uniformly distanced quantiles.
This was the official measure used in the M5 Uncertainty competition
with seasonality = 1.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions for each quantile. |
| quantiles | ndarray |  | Quantiles to compare against. |
| seasonality | int |  | Main frequency of the time series;<br>Hourly 24, Daily 7, Weekly 52, Monthly 12, Quarterly 4, Yearly 1. |
| train_df | DFType |  | Training dataframe with id and actual values. Must be sorted by time. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    scaled_mqloss(series, mq_models, quantiles=quantiles, seasonality=1, train_df=series),
    scaled_mqloss(series_pl, mq_models, quantiles=quantiles, seasonality=1, train_df=series_pl),
    models,
)

### Coverage

In [ ]:
#| export
def coverage(
    df: DFType,
    models: List[str],
    level: int,
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Coverage of y with y_hat_lo and y_hat_hi.

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : list of str
        Columns that identify the models predictions.
    level : int
        Confidence level used for intervals.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://www.jstor.org/stable/2629907        
    """
    if isinstance(df, pd.DataFrame):
        out = np.empty((df.shape[0], len(models)))
        for j, model in enumerate(models):
            out[:, j] = df[target_col].between(df[f'{model}-lo-{level}'], df[f'{model}-hi-{level}'])
        res = pd.DataFrame(
            out, columns=models, index=df.index
        ).groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        def gen_expr(model):
            return pl.col(target_col).is_between(pl.col(f'{model}-lo-{level}'), pl.col(f'{model}-hi-{level}')).alias(model)

        res = _pl_agg_expr(df, models, id_col, gen_expr)
    return res

In [ ]:
show_doc(coverage, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L762){target="_blank" style="float:right; font-size:smaller"}

#### coverage

>      coverage (df:~DFType, models:List[str], level:int,
>                id_col:str='unique_id', target_col:str='y')

*Coverage of y with y_hat_lo and y_hat_hi.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, times, actuals and predictions. |
| models | List |  | Columns that identify the models predictions. |
| level | int |  | Confidence level used for intervals. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    coverage(series, models, 80),
    coverage(series_pl, models, 80),
    models,
)

### Calibration

In [ ]:
#| export
def calibration(
    df: DFType,
    models: Dict[str, str],
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """
    Fraction of y that is lower than the model's predictions. 
    
    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to str
        Mapping from model name to the model predictions.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.
        
    References
    ----------
    [1] https://www.jstor.org/stable/2629907            
    """
    if isinstance(df, pd.DataFrame):
        out = np.empty((df.shape[0], len(models)))
        for j, q_preds in enumerate(models.values()):
            out[:, j] = df[target_col].le(df[q_preds])
        res = pd.DataFrame(out, columns=models.keys(), index=df.index).groupby(df[id_col], observed=True).mean()
        res.index.name = id_col
        res = res.reset_index()
    else:
        def gen_expr(model):
            model_name, q_preds = model
            return pl.col(target_col).le(pl.col(q_preds)).alias(model_name)

        res = _pl_agg_expr(df, list(models.items()), id_col, gen_expr)
    return res    

In [ ]:
show_doc(calibration, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L821){target="_blank" style="float:right; font-size:smaller"}

#### calibration

>      calibration (df:~DFType, models:Dict[str,str], id_col:str='unique_id',
>                   target_col:str='y')

*Fraction of y that is lower than the model's predictions.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    calibration(series, q_models[0.1]),
    calibration(series_pl, q_models[0.1]),
    models,
)

### CRPS

$$
\mathrm{sCRPS}(\hat{F}_{\tau}, \mathbf{y}_{\tau}) = \frac{2}{N} \sum_{i}
\int^{1}_{0} \frac{\mathrm{QL}(\hat{F}_{i,\tau}, y_{i,\tau})_{q}}{\sum_{i} | y_{i,\tau} |} dq
$$

Where $\hat{F}_{\tau}$ is the an estimated multivariate distribution, and $y_{i,\tau}$ are its realizations. 

In [ ]:
#| export
def scaled_crps(
    df: DFType,
    models: Dict[str, List[str]],
    quantiles: np.ndarray,
    id_col: str = 'unique_id',
    target_col: str = 'y',
) -> DFType:
    """Scaled Continues Ranked Probability Score
    
    Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
    to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.
    This metric averages percentual weighted absolute deviations as 
    defined by the quantile losses.
    
    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, times, actuals and predictions.
    models : dict from str to list of str
        Mapping from model name to the model predictions for each quantile.
    quantiles : numpy array
        Quantiles to compare against.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars Dataframe
        dataframe with one row per id and one column per model.

    References
    ----------
    [1] https://proceedings.mlr.press/v139/rangapuram21a.html        
    """
    eps: np.float64 = np.finfo(np.float64).eps
    quantiles = np.asarray(quantiles)
    loss = mqloss(df, models, quantiles, id_col, target_col)
    sizes = ufp.counts_by_id(df, id_col)
    if isinstance(loss, pd.DataFrame):
        loss = loss.set_index(id_col)
        sizes = sizes.set_index(id_col)
        assert isinstance(df, pd.DataFrame)
        norm = df[target_col].abs().groupby(df[id_col], observed=True).sum()
        res = 2 * loss.mul(sizes['counts'], axis=0).div(norm + eps, axis=0)
        res.index.name = id_col
        res = res.reset_index()
    else:
        def gen_expr(model):
            return (2 * pl.col(model) * pl.col('counts') / (pl.col('norm') + eps)).alias(model)

        grouped_df = ufp.group_by(df, id_col)
        norm = grouped_df.agg(pl.col(target_col).abs().sum().alias('norm'))
        res = _pl_agg_expr(
            loss.join(sizes, on=id_col).join(norm, on=id_col),
            list(models.keys()),
            id_col,
            gen_expr,
        )
    return res

In [ ]:
show_doc(scaled_crps, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L871){target="_blank" style="float:right; font-size:smaller"}

#### scaled_crps

>      scaled_crps (df:~DFType, models:Dict[str,List[str]],
>                   quantiles:numpy.ndarray, id_col:str='unique_id',
>                   target_col:str='y')

*Scaled Continues Ranked Probability Score

Calculates a scaled variation of the CRPS, as proposed by Rangapuram (2021),
to measure the accuracy of predicted quantiles `y_hat` compared to the observation `y`.
This metric averages percentual weighted absolute deviations as 
defined by the quantile losses.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, times, actuals and predictions. |
| models | Dict |  | Mapping from model name to the model predictions for each quantile. |
| quantiles | ndarray |  | Quantiles to compare against. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    scaled_crps(series, mq_models, quantiles),
    scaled_crps(series_pl, mq_models, quantiles),
    models,
)

### Tweedie Deviance

For a set of forecasts $\{\mu_i\}_{i=1}^N$ and observations $\{y_i\}_{i=1}^N$, the mean Tweedie deviance with power $p$ is

$$
\mathrm{TD}_{p}(\boldsymbol{\mu}, \mathbf{y})
= \frac{1}{N} \sum_{i=1}^{N} d_{p}(y_i, \mu_i)
$$

where the unit-scaled deviance for each pair $(y,\mu)$ is

$$
d_{p}(y,\mu)
=
2
\begin{cases}
\displaystyle
\frac{y^{2-p}}{(1-p)(2-p)}
\;-\;
\frac{y\,\mu^{1-p}}{1-p}
\;+\;
\frac{\mu^{2-p}}{2-p}, 
& p \notin\{1,2\},\\[1em]
\displaystyle
y\,\ln\!\frac{y}{\mu}\;-\;(y-\mu),
& p = 1\quad(\text{Poisson deviance}),\\[0.5em]
\displaystyle
-2\Bigl[\ln\!\frac{y}{\mu}\;-\;\frac{y-\mu}{\mu}\Bigr],
& p = 2\quad(\text{Gamma deviance}).
\end{cases}
$$

- $y_i$ are the true values, $\mu_i$ the predicted means.  
- $p$ controls the variance relationship $\mathrm{Var}(Y)\propto\mu^{p}$.  
- When $1<p<2$, this smoothly interpolates between Poisson ($p=1$) and Gamma ($p=2$) deviance.


In [ ]:
#| export
def mean_tweedie_deviance(y_true: ArrayLike, y_pred: ArrayLike, power: float):
    """
    Compute the average Tweedie deviance between true values and predictions.

    The Tweedie deviance is defined differently depending on the power parameter:
      - power = 0: equivalent to mean squared error.
      - power = 1: equivalent to mean Poisson deviance.
      - power = 2: equivalent to mean gamma deviance.
      - other powers: general Tweedie deviance.

    Parameters
    ----------
    y_true : array-like
        Ground truth (correct) target values. Must be convertible to a NumPy array of floats.
    y_pred : array-like
        Predicted target values. Must be convertible to a NumPy array of floats and strictly positive.
    power : float
        Tweedie power parameter. Determines the distribution:
        0 for normal, 1 for Poisson, 2 for gamma, else general.
    
    Returns
    -------
    float
        Average Tweedie deviance over all samples
    """
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)

    if np.any(y_pred <= 0):
        raise ValueError("All predictions must be strictly positive for Tweedie deviance.")

    if power == 0:
        dev = (y_true - y_pred) ** 2

    elif power == 1:
        with np.errstate(divide='ignore', invalid='ignore'):
            dev = 2 * (y_true * np.log(y_true / y_pred) - (y_true - y_pred))
        zero_mask = (y_true == 0)
        dev[zero_mask] = 2 * y_pred[zero_mask]

    elif power == 2:
        dev = 2 * (-np.log(y_true / y_pred) + (y_true / y_pred) - 1)

    else:
        dev = 2 * (
            y_true**(2 - power) / ((1 - power) * (2 - power))
            - y_true * y_pred**(1 - power) / (1 - power)
            + y_pred**(2 - power) / (2 - power)
        )

    return np.mean(dev)

In [ ]:
show_doc(mean_tweedie_deviance, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L937){target="_blank" style="float:right; font-size:smaller"}

#### mean_tweedie_deviance

>      mean_tweedie_deviance (y_true:Union[numpy._typing._array_like._Buffer,num
>                             py._typing._array_like._SupportsArray[numpy.dtype[
>                             Any]],numpy._typing._nested_sequence._NestedSequen
>                             ce[numpy._typing._array_like._SupportsArray[numpy.
>                             dtype[Any]]],bool,int,float,complex,str,bytes,nump
>                             y._typing._nested_sequence._NestedSequence[bool|in
>                             t|float|complex|str|bytes]], y_pred:Union[numpy._t
>                             yping._array_like._Buffer,numpy._typing._array_lik
>                             e._SupportsArray[numpy.dtype[Any]],numpy._typing._
>                             nested_sequence._NestedSequence[numpy._typing._arr
>                             ay_like._SupportsArray[numpy.dtype[Any]]],bool,int
>                             ,float,complex,str,bytes,numpy._typing._nested_seq
>                             uence._NestedSequence[bool|int|float|complex|str|b
>                             ytes]], power:float)

*Compute the average Tweedie deviance between true values and predictions.

The Tweedie deviance is defined differently depending on the power parameter:
  - power = 0: equivalent to mean squared error.
  - power = 1: equivalent to mean Poisson deviance.
  - power = 2: equivalent to mean gamma deviance.
  - other powers: general Tweedie deviance.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| y_true | Union | Ground truth (correct) target values. Must be convertible to a NumPy array of floats. |
| y_pred | Union | Predicted target values. Must be convertible to a NumPy array of floats and strictly positive. |
| power | float | Tweedie power parameter. Determines the distribution:<br>0 for normal, 1 for Poisson, 2 for gamma, else general. |
| **Returns** | **float** | **Average Tweedie deviance over all samples** |

In [ ]:
#| export
def tweedie_deviance(
    df: DFType,
    models: List[str],
    power: float = 1.5,  
    id_col: str = "unique_id",
    target_col: str = "y",
) -> DFType:
    """Compute the Tweedie deviance loss for one or multiple models, grouped by an identifier.

    Each group's deviance is calculated using the mean_tweedie_deviance function, which
    measures the deviation between actual and predicted values under the Tweedie distribution.

    The power parameter defines the specific compound distribution:
      - 1: Poisson
      - (1, 2): Compound Poisson-Gamma
      - 2: Gamma
      - >2: Inverse Gaussian

    Parameters
    ----------
    df : pandas or polars DataFrame
        Input dataframe with id, actual values and predictions.
    models : list of str
        Columns that identify the models predictions.
    power : float, optional (default=1.5)
        Tweedie power parameter defining the distribution.
    id_col : str (default='unique_id')
        Column that identifies each serie.
    target_col : str (default='y')
        Column that contains the target.

    Returns
    -------
    pandas or polars DataFrame
        dataframe with one row per id and one column per model.
    """
    if isinstance(df, pd.DataFrame):
        res = (
            df.groupby(id_col, observed=True)
            .apply(lambda g: {
                model: mean_tweedie_deviance(
                    y_true=g[target_col],
                    y_pred=g[model],
                    power=power
                )
                for model in models
            })
            .apply(pd.Series)
            .reset_index()
        )
    else:
        def gen_expr(model):
            return pl.struct([target_col, model]).map_elements(
                lambda s: mean_tweedie_deviance(
                    y_true=[s[target_col]],
                    y_pred=[s[model]],
                    power=power
                ),
                return_dtype=pl.Float64
            ).alias(model)

        res = _pl_agg_expr(df, models, id_col, gen_expr)
    return res


In [ ]:
show_doc(tweedie_deviance, title_level=4)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/losses.py#L998){target="_blank" style="float:right; font-size:smaller"}

#### tweedie_deviance

>      tweedie_deviance (df:~DFType, models:List[str], power:float=1.5,
>                        id_col:str='unique_id', target_col:str='y')

*Compute the Tweedie deviance loss for one or multiple models, grouped by an identifier.

Each group's deviance is calculated using the mean_tweedie_deviance function, which
measures the deviation between actual and predicted values under the Tweedie distribution.

The power parameter defines the specific compound distribution:
  - 1: Poisson
  - (1, 2): Compound Poisson-Gamma
  - 2: Gamma
  - >2: Inverse Gaussian*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df | DFType |  | Input dataframe with id, actual values and predictions. |
| models | List |  | Columns that identify the models predictions. |
| power | float | 1.5 | Tweedie power parameter defining the distribution. |
| id_col | str | unique_id | Column that identifies each serie. |
| target_col | str | y | Column that contains the target. |
| **Returns** | **DFType** |  | **dataframe with one row per id and one column per model.** |

In [ ]:
#| polars
pd_vs_pl(
    tweedie_deviance(series,   models, target_col="y", power=1.5),
    tweedie_deviance(series_pl,models, target_col="y", power=1.5),
    models,
)

/var/folders/g1/xrd363zx571_wq54f3htnbxr0000gn/T/ipykernel_8131/2024453147.py:41: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: {
